# This is a document that tries keras on jupyter notebook
# I attempted to use letter by letter plus RNN to inteprete description, but it seems to be not good
# possible reasons including too small database, too difficult the task.
# I am starting another atempt through word base classification


In [37]:
import numpy as np
import pandas as pd
import gensim
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import GaussianNoise
from keras.layers import LSTM
from keras.layers import Activation
from keras.optimizers import Adam
from keras.regularizers import l1, l2
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import Callback, LambdaCallback, TensorBoard, ReduceLROnPlateau
from keras.utils import np_utils

In [2]:
def is_desired_letter(char):
    return ord(char) >= 97 and ord(char) < 123 or ord(char) >= 48 and ord(char) < 58 or ord(char) == ord(".") or ord(char) == ord(",") or ord(char) == ord(" ")


def get_train_data(train_portion):
    # load the dataset but only keep the top n words, zero the rest
    train_data = pd.read_csv("input/kickstarter_train.csv")
    train_texts_and_results = train_data.iloc[:, [2, -1]]
    # get split point for train and test data
    split_point = int(train_portion * len(train_data))
    # do preliminary preprocessing:remove all symbols
    train_data["desc"] = [[char for char in str(text).lower() if is_desired_letter(char)] for
                          text in train_data["desc"]]
    # remove too short desc
    drop_index = []
    for i in range(len(train_data)):
        if len(train_data.iloc[i, 2]) <= 20:
            drop_index.append(i)
    train_data.drop(train_data.index[drop_index])
    # get descriptions data
    train_texts = np.array(train_data.iloc[:split_point, 2])
    test_texts = np.array(train_data.iloc[split_point:, 2])
    # get num data
    train_num = np.array(train_data.iloc[:split_point, [3, 12]])
    test_num = np.array(train_data.iloc[split_point:, [3, 12]])
    # get result data
    train_results = np.array(train_data.iloc[:split_point, -1])
    test_results = np.array(train_data.iloc[split_point:, -1])
    
    return train_texts, train_num, train_results, test_texts, test_num, test_results


def convert_to_onehot(data, num_features):
    new_data = []
    for item in data:
        new_data.append(np_utils.to_categorical(item, num_classes=num_features))
    return np.array(new_data)

In [3]:
# get training testing data from disk
train_data_portion = 0.95
trainX_desc, trainX_num, trainY, testX_desc, testX_num, testY = get_train_data(train_data_portion)
print("data grabbed")

# convert char to int, and 
all_letters = sorted(set([char for text in trainX_desc for char in text]))
n_vacab = len(all_letters)
char_to_int = dict((c, float(i+1)) for i, c in enumerate(all_letters))
trainX_desc = [[char_to_int[char] * 2 / float(n_vacab) - 1 for char in text] for text in trainX_desc]
testX_desc = [[char_to_int[char] * 2 / float(n_vacab) - 1 for char in text] for text in testX_desc]
print("tokenizing and normalizing is done")

data grabbed
tokenizing and normalizing is done


In [5]:
# preprocessing description data
# truncate and pad input sequences
max_desc_length = 200
trainX_desc = sequence.pad_sequences(list(trainX_desc), maxlen=max_desc_length, truncating="post")
testX_desc = sequence.pad_sequences(list(testX_desc), maxlen=max_desc_length, truncating="post")
print("padding finished")

# reshape trainX to multi_timestep single feature
time_steps = max_desc_length
num_features = 1
testX_desc = np.array(testX_desc)
testX_desc = testX_desc.reshape((-1, time_steps, num_features))
trainX_desc = np.array(trainX_desc)
trainX_desc = trainX_desc.reshape((-1, time_steps, num_features))
print("reshaping data with shape {}".format(trainX_desc.shape))

padding finished
reshaping data with shape (86503, 200, 1)


In [26]:
class My_Callback(Callback):
    def on_epoch_begin(self, logs={}):
        scores = self.model.evaluate(self.model.validation_data[0], self.model.validation_data[1], verbose=1)
        print("Accuracy on Validation data:{}".format(np.array(scores).mean()))
        return

In [38]:
# generate model for descriptions
description_model = Sequential()
description_model.add(LSTM(64, input_shape=(time_steps, num_features), return_sequences=True))
description_model.add(LSTM(64))
description_model.add(Dropout(0.2))
description_model.add(Dense(units=64, activation="sigmoid"))
description_model.add(BatchNormalization())
description_model.add(Dropout(0.5))
description_model.add(Dense(1, activation='sigmoid'))

# configurate model training
description_model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
print("model building finished\n", description_model.summary())


# do training
validation_callback = My_Callback()
description_model.fit(trainX_desc, trainY, epochs=10, batch_size=256, shuffle=True, validation_data=[testX_desc, testY], 
                      verbose=1, callbacks=[TensorBoard(), ReduceLROnPlateau()])

# see actual result
scores = description_model.evaluate(testX_desc, testY, verbose=1)
print("Accuracy:{}".format(np.array(scores).mean()))

# save model
filepath = "description_model_weights.h5"
description_model.save_weights(filepath)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 200, 64)           16896     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 64)                4160      
_________________________________________________________________
batch_normalization_34 (Batc (None, 64)                256       
_________________________________________________________________
dropout_23 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 65        
Total para

KeyboardInterrupt: 

In [19]:
description_model.load_weights(filepath)

# do training
description_model.fit(trainX_desc, trainY, epochs=10, batch_size=256, shuffle=True, validation_data=[testX_desc, testY], verbose=1)

# see original result
scores = description_model.evaluate(testX_desc, testY, verbose=1)
print("Accuracy:{}".format(np.array(scores).mean()))

Train on 86503 samples, validate on 21626 samples
Epoch 1/10
 - 28s - loss: 0.6431 - acc: 0.6571 - val_loss: 0.5618 - val_acc: 0.7732
Epoch 2/10
 - 28s - loss: 0.6431 - acc: 0.6570 - val_loss: 0.5801 - val_acc: 0.7732
Epoch 3/10
 - 29s - loss: 0.6431 - acc: 0.6571 - val_loss: 0.5705 - val_acc: 0.7732
Epoch 4/10
 - 28s - loss: 0.6430 - acc: 0.6571 - val_loss: 0.5586 - val_acc: 0.7732
Epoch 5/10
 - 29s - loss: 0.6431 - acc: 0.6572 - val_loss: 0.5793 - val_acc: 0.7732
Epoch 6/10
 - 30s - loss: 0.6430 - acc: 0.6572 - val_loss: 0.5500 - val_acc: 0.7732
Epoch 7/10
 - 28s - loss: 0.6431 - acc: 0.6571 - val_loss: 0.5464 - val_acc: 0.7732
Epoch 8/10
 - 28s - loss: 0.6430 - acc: 0.6571 - val_loss: 0.5721 - val_acc: 0.7732
Epoch 9/10
 - 31s - loss: 0.6431 - acc: 0.6572 - val_loss: 0.5711 - val_acc: 0.7732
Epoch 10/10
 - 28s - loss: 0.6430 - acc: 0.6572 - val_loss: 0.5671 - val_acc: 0.7732
21626/21626 [==============================] - 8s 367us/step
Accuracy:0.6701744736208347


In [ ]:
# generate model for num data
num_model = Sequential()
num_model.add(Dense(units=64, input_shape=(2)))
num_model.add(BatchNormalization())
num_model.add(Activation("sigmoid"))
num_model.add(Dropout(0.2))
num_model.add(Dense(units=64))
num_model.add(BatchNormalization())
num_model.add(Dropout(0.2))